import


In [1]:
import torch
import torch.nn as nn
from vilt.modules import heads, objectives
import vilt.modules.vision_transformer as vit
import vilt.modules.vision_transformer as vit
import torch.nn.functional as F
import random
from typing import OrderedDict
import os
import pandas as pd
import numpy as np
from vilt.transforms import pixelbert_transform
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import wandb




config


In [2]:


class config:
    debug = False
    sensor_only = False
    crop_name = "total"

    
    exp_name = "vilt"
    seed = 101
    batch_size = 4096  # this is a desired batch size; pl trainer will accumulate gradients when per step batch is smaller.
    train_batch_size = 32
    valid_batch_size = 4
    device = "cpu"
    n_fold = 5

    model_name = "sensorViLOnlyTransformerSS" #仅图片
    # model_name = "sensorOnlyViLTransformerSS"  #仅vilt传感器
    # model_name = "sensorViLTransformerSS"  #vilt图像+传感器
    # model_name = "DNNF1"  #DNNF1图像+传感器
    # model_name = "DNNF1PictureOnly"  #DNNF1图像
    # model_name = "DNNF1SensorOnly"  #DNNF1传感器

    # model_name = "DNNF2"  #DNNF2图像+传感器
    # model_name = "DNNF2PictureOnly"  #DNNF1图像
    # model_name = "DNNF2SensorOnly"  #DNNF1传感器
    # wandb 
    # wandb_name = "vilt|大豆|290图像加传感器"
    # wandb_name = "vilt|大豆|290仅传感器"
    wandb_name = "vilt|大豆|290仅图片"

    # wandb_name = "DNNF1|大豆|290图像加传感器"
    # wandb_name = "DNNF1|大豆|290仅图像"
    # wandb_name = "DNNF1|大豆|290仅传感器"
    
    # wandb_name = "DNNF2|大豆|290图像加传感器"
    # wandb_name = "DNNF2|大豆|290仅图像"
    # wandb_name = "DNNF2|大豆|290仅传感器"
    

    # Image setting
    train_transform_keys = ["pixelbert"]
    val_transform_keys = ["pixelbert"]
    img_size = 384
    max_image_len = -1
    patch_size = 32
    draw_false_image = 1
    image_only = False

    # Sensor
    # senser_input_num = 11 # 翔冠的传感器参数
    senser_input_num = 19 # 天航的传感器参数
    
    # Text Setting
    vqav2_label_size = 3129
    max_text_len = 40
    tokenizer = "bert-base-uncased"
    vocab_size = 30522 # vocabulary词汇数量
    whole_word_masking = False
    mlm_prob = 0.15
    draw_false_text = 0

    # Transformer Setting
    vit = "vit_base_patch32_384"
    hidden_size = 768  # 嵌入向量大小
    num_heads = 12
    num_layers = 12
    mlp_ratio = 4
    drop_rate = 0.1

    # Optimizer Setting
    optim_type = "adamw"
    learning_rate = 1e-3 #0.0015#2e-3 #
    weight_decay = 1e-4 # 0.01 ->1e-4
    decay_power = 1
    max_epoch = 50
    max_steps = 25000
    warmup_steps = 2500
    end_lr = 0
    lr_mult = 1  # multiply lr for downstream heads
    # T_max = 8000/train_batch_size*max_epoch 
    T_max = 4632/train_batch_size*max_epoch 

    # Downstream Setting
    get_recall_metric = False


    # below params varies with the environment
    data_root = ""
    log_dir = "result"
    per_gpu_batchsize = 0  # you should define this manually with per_gpu_batch_size=#
    num_gpus = 1
    num_nodes = 1
    load_path = "weights/vilt_200k_mlm_itm.ckpt"
    # load_path = "save_model_dict.pt"
    num_workers = 1
    precision = 16

# config = vars(config)
# config = dict(config)
config

if config.debug:
    config.max_epoch = 5

model

In [3]:
l1 = nn.Linear(1,145) 
l2 = nn.Linear(19,768)
z = torch.rand([2,1,19])
z = l2(z) #2,1,768
z = torch.transpose(z,1,2)
z = l1(z)
z=torch.transpose(z,1,2)

In [4]:

class ViST(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 
        self.sensor_linear2 = nn.Linear(1,145)

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        # self.transformer = getattr(vit, config.vit)(
        #         pretrained=False, config=vars(config)
        #     )
        self.transformer = vit.VisionTransformerForViST(img_size=config.img_size,patch_size=config.patch_size,embed_dim=config.hidden_size,num_heads=config.num_heads,mlp_ratio=config.mlp_ratio,qkv_bias=False,qk_scale=None)
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        sensor_embeds = torch.transpose(sensor_embeds,1,2)
        sensor_embeds = self.sensor_linear2(sensor_embeds)
        sensor_embeds = torch.transpose(sensor_embeds,1,2)

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)
            (
                image_embeds, # torch.Size([1, 217, 768])
                image_masks, # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        
        x_image = image_embeds # torch.Size([2, 145, 768])
        x_sensor = sensor_embeds # torch.Size([2, 145, 768])
        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x_image,x_sensor, _attn_image,_attn_sensor = blk(x_image, x_sensor) # co_masks = torch.Size([1, 211])

        x = torch.cat([x_image, x_sensor], dim=1)
        # x = self.transformer.norm(x) # torch.Size([1, 240, 768])

        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        
        ret = {
           "sensor_feats":x_image,
            "image_feats": x_sensor,
            "cls_feats": cls_feats, # class features
            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


test data

In [5]:
model = ViST(sensor_class_n= config.senser_input_num,output_class_n = 1)

In [6]:
sensor = torch.rand([2,1,19])
# sensor = torch.ones(config.senser_input_num)

# sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])
print(sensor.shape)
batch = {}
batch['sensor'] = sensor
batch['image'] = torch.randn([2,3,384,384])
model(batch)

torch.Size([2, 1, 19])


{'sensor_feats': tensor([[[ 0.6595, -0.4498,  0.3922,  ...,  0.0204, -0.1160, -0.2402],
          [ 0.6246, -0.7694,  0.0090,  ...,  0.0950,  0.1712,  0.2680],
          [ 0.3803, -0.8262,  0.3992,  ..., -0.2846,  0.2506, -0.0940],
          ...,
          [ 0.9158, -1.2279, -0.0145,  ...,  0.2157, -0.4875, -0.0879],
          [ 1.1979, -1.0235,  0.1862,  ...,  0.3160, -0.2873,  0.0644],
          [ 1.1956, -0.6469,  0.4673,  ...,  0.0361,  0.3572, -0.0267]],
 
         [[ 1.0390, -0.3722, -0.0572,  ..., -0.2879, -0.3757,  0.1105],
          [ 0.1092,  0.0326, -0.2876,  ...,  0.0239,  0.2742, -0.5106],
          [ 0.3016, -0.3207, -0.1068,  ...,  0.3272, -0.1817,  0.3776],
          ...,
          [ 0.8873,  0.0691, -0.4358,  ...,  0.2184, -0.3868, -0.0499],
          [ 0.4286,  0.2746, -0.0911,  ..., -0.2071,  0.0686, -0.1166],
          [ 0.7101,  0.1664, -0.4994,  ...,  0.0935,  0.1184,  0.3041]]],
        grad_fn=<AddBackward0>),
 'image_feats': tensor([[[ 0.0897,  0.3390, -0.4257,

In [7]:
attn = torch.randn([2,12,146,146])
v = torch.randn([2,12,146,64])
(attn @ v).shape

torch.Size([2, 12, 146, 64])

test for attention

In [8]:
model_attn = vit.AttentionForViST(dim=768,num_heads=12,qkv_bias=False,qk_scale=None,attn_drop=0.0,proj_drop=0.0)

In [9]:
image_input = torch.randn([2,145,768])
sensor_input = torch.randn([2,145,768])

In [10]:
x_image,x_sensor, attn_image,attn_sensor = model_attn(image_input,sensor_input)

In [11]:
model_vist = vit.VisionTransformerForViST(img_size=config.img_size,patch_size=config.patch_size,embed_dim=config.hidden_size,num_heads=config.num_heads,mlp_ratio=config.mlp_ratio,qkv_bias=False,qk_scale=None)
